Import Libs

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from models import model_sfnn_train
from models import model_rf_train, model_naives_bayes, model_logistic_regression, predict_values, model_xgboost



d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importing and reading the data

In [3]:
# get csv data
def read_Data(path):

    """
    Read a tab-delimited text file and return a cleaned DataFrame with labels and text.

    This function reads a CSV/TSV file without headers, assigns column names
    ['label', 'text'], removes potential byte-order marks (BOM) from column names
    and label values, and ensures that the 'label' column is of integer type.

    Parameters
    ----------
    path : str
        Path to the tab-delimited text file to read.

    Returns
    -------
    df : pandas.DataFrame
        A DataFrame with two columns:
        - 'label' : int, the class label for each text sample
        - 'text' : str, the text data

    """
    # Define column names
    column_names = ['label', 'text']

    df = pd.read_csv(path, delimiter='\t', encoding="utf-8-sig", header=None, names=column_names)
    df.columns = df.columns.str.replace('\ufeff', '')
    df['label'] = df['label'].astype(str).str.replace('\ufeff', '')
    df['label'] = df['label'].astype(int)
    
    return df



In [4]:
## read data windows
#data_train_val = read_Data(r".\data\training_data_lowercase.csv")           # used for train and val data
#data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")        # data for the predictions

## read data Mac
data_train_val = read_Data("data/training_data_lowercase.csv")           # used for train and val data
data_test = read_Data("data/testing_data_lowercase_nolabels.csv")        # data for the predictions


## split data_train into train and val data
data_train, data_val = train_test_split(data_train_val, test_size=0.08, random_state=42, stratify=data_train_val['label'])


In [5]:
## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train_val.shape)
print("Shape test data:\n", data_test.shape)


Shape train data:
 (34152, 2)
Shape test data:
 (9984, 2)


## Pre-processing data

Setup with advanced data cleaning

In [6]:
# Preprocessing steps

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

#print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
#print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

#print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
#print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")

# calc tf-idf matrix on lemmatized data
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf_matrix_train = vectorizer.fit_transform(data_train['lemmatized_text'])
tfidf_matrix_val = vectorizer.transform(data_val['lemmatized_text'])
tfidf_matrix_test = vectorizer.transform(data_test['lemmatized_text'])


**TF-IDF only**

With and without stop-words function

Pre-processing setup 2: TF-IDF Vectorizer (without stopwords argument)

In [7]:
# calc tf-idf matrix with uncleaned data
vectorizer2 = TfidfVectorizer(ngram_range=(1, 3))
X_train_tfidf = vectorizer2.fit_transform(data_train['text'])
X_val_tfidf = vectorizer2.transform(data_val['text'])
X_test_tfidf = vectorizer2.transform(data_test['text'])


Pre-processing setup 3: TF-IDF Vectorizer (without stopwords argument)

In [8]:

# calc tf-idf matrix with uncleaned data, tf-idf removing stopwords
vectorizer3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 3))
X_train_tfidf3 = vectorizer3.fit_transform(data_train['text'])
X_val_tfidf3 = vectorizer3.transform(data_val['text'])
X_test_tfidf3 = vectorizer3.transform(data_test['text'])

Pre-processing setup 4: Embeddings

In [9]:
# Load pretrained embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text to embeddings
X_train_emb = embedder.encode(data_train['text'].tolist(), show_progress_bar=True)
X_val_emb = embedder.encode(data_val['text'].tolist(), show_progress_bar=True)
X_test_emb = embedder.encode(data_test['text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 312/312 [00:17<00:00, 17.52it/s]


## Applying and evaluating models

### Naive Bayes Model

In [9]:
# Preprocessing setup 1: calc tf-idf on lemmatized data
model_nb1 = model_naives_bayes(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Train Accuracy:1.00

Validation Accuracy: 0.94

classification_report:
               precision    recall  f1-score   support

           0     0.9390    0.9531    0.9460      1406
           1     0.9495    0.9344    0.9419      1327

    accuracy                         0.9440      2733
   macro avg     0.9442    0.9437    0.9439      2733
weighted avg     0.9441    0.9440    0.9440      2733



In [10]:
# Preprocessing setup 2: tf-idf only
model_nb2 = model_naives_bayes(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])


Train Accuracy:1.00

Validation Accuracy: 0.95

classification_report:
               precision    recall  f1-score   support

           0     0.9383    0.9623    0.9501      1406
           1     0.9589    0.9329    0.9458      1327

    accuracy                         0.9480      2733
   macro avg     0.9486    0.9476    0.9480      2733
weighted avg     0.9483    0.9480    0.9480      2733



In [11]:
# Preprocessing setup 3: tf-idf removing stopwords
model_nb3 = model_naives_bayes(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])


Train Accuracy:1.00

Validation Accuracy: 0.94

classification_report:
               precision    recall  f1-score   support

           0     0.9350    0.9516    0.9432      1406
           1     0.9478    0.9299    0.9388      1327

    accuracy                         0.9411      2733
   macro avg     0.9414    0.9408    0.9410      2733
weighted avg     0.9412    0.9411    0.9411      2733



### XGBoost

In [12]:
# XGBoost Setup 1
model_xgb1 = model_xgboost(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

Train Accuracy: 0.90

Validation Accuracy: 0.84

Classification Report:
               precision    recall  f1-score   support

           0     0.8827    0.7973    0.8378      1406
           1     0.8052    0.8877    0.8444      1327

    accuracy                         0.8412      2733
   macro avg     0.8439    0.8425    0.8411      2733
weighted avg     0.8451    0.8412    0.8410      2733



In [13]:
# XGBoost Setup 2
model_xgb2 = model_xgboost(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])

Train Accuracy: 0.93

Validation Accuracy: 0.83

Classification Report:
               precision    recall  f1-score   support

           0     0.8162    0.8620    0.8385      1406
           1     0.8446    0.7943    0.8186      1327

    accuracy                         0.8291      2733
   macro avg     0.8304    0.8281    0.8286      2733
weighted avg     0.8299    0.8291    0.8288      2733



In [14]:
# XGBoost Setup 3
model_xgb3 = model_xgboost(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])

Train Accuracy: 0.90

Validation Accuracy: 0.82

Classification Report:
               precision    recall  f1-score   support

           0     0.8192    0.8378    0.8284      1406
           1     0.8239    0.8041    0.8139      1327

    accuracy                         0.8214      2733
   macro avg     0.8216    0.8210    0.8211      2733
weighted avg     0.8215    0.8214    0.8214      2733



### Random Forest

In [15]:
# Random Forest setup 1
model_rf1 = model_rf_train(tfidf_matrix_train, tfidf_matrix_val, data_train['label'], data_val['label'])

Train Accuracy: 100.00%
Random forest model
accuracy: 0.9063300402488108
Classification report:
               precision    recall  f1-score   support

           0       0.91      0.90      0.91      1406
           1       0.90      0.91      0.90      1327

    accuracy                           0.91      2733
   macro avg       0.91      0.91      0.91      2733
weighted avg       0.91      0.91      0.91      2733



In [16]:
# Random Forest setup 2
model_rf2 = model_rf_train(X_train_tfidf, X_val_tfidf, data_train['label'], data_val['label'])

Train Accuracy: 100.00%
Random forest model
accuracy: 0.9085254299304794
Classification report:
               precision    recall  f1-score   support

           0       0.93      0.89      0.91      1406
           1       0.89      0.93      0.91      1327

    accuracy                           0.91      2733
   macro avg       0.91      0.91      0.91      2733
weighted avg       0.91      0.91      0.91      2733



In [17]:
# Random Forest setup 2
model_rf3 = model_rf_train(X_train_tfidf3, X_val_tfidf3, data_train['label'], data_val['label'])

Train Accuracy: 100.00%
Random forest model
accuracy: 0.9008415660446396
Classification report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91      1406
           1       0.92      0.87      0.90      1327

    accuracy                           0.90      2733
   macro avg       0.90      0.90      0.90      2733
weighted avg       0.90      0.90      0.90      2733



### Logistic Regression

In [18]:
#Logistic Regression setup1
model_lr1 = model_logistic_regression(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Train Accuracy: 96.91%

Validation Accuracy: 93.27%

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      1406
           1       0.92      0.94      0.93      1327

    accuracy                           0.93      2733
   macro avg       0.93      0.93      0.93      2733
weighted avg       0.93      0.93      0.93      2733



In [19]:
#Logistic Regression setup2
model_lr2 = model_logistic_regression(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])


Train Accuracy: 97.62%

Validation Accuracy: 94.04%

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      1406
           1       0.94      0.94      0.94      1327

    accuracy                           0.94      2733
   macro avg       0.94      0.94      0.94      2733
weighted avg       0.94      0.94      0.94      2733



In [20]:
#Logistic Regression setup3
model_lr3 = model_logistic_regression(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])


Train Accuracy: 97.54%

Validation Accuracy: 92.54%

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      1406
           1       0.91      0.94      0.92      1327

    accuracy                           0.93      2733
   macro avg       0.93      0.93      0.93      2733
weighted avg       0.93      0.93      0.93      2733



In [21]:
# Preprocessing setup 4: Embeddings
model_lr4 = model_logistic_regression(X_train_emb, data_train['label'], X_val_emb, data_val['label'])


Train Accuracy: 92.98%

Validation Accuracy: 92.61%

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.90      0.93      1406
           1       0.90      0.95      0.93      1327

    accuracy                           0.93      2733
   macro avg       0.93      0.93      0.93      2733
weighted avg       0.93      0.93      0.93      2733



### Simple Feedforward Neural Network

In [22]:
# Simple Feedforward NN with preprocessing setup1
model_sfnn1 = model_sfnn_train(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8551 - loss: 0.3242 - val_accuracy: 0.9308 - val_loss: 0.1810
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9364 - loss: 0.1592 - val_accuracy: 0.9330 - val_loss: 0.1733
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9533 - loss: 0.1213 - val_accuracy: 0.9356 - val_loss: 0.1685
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9689 - loss: 0.0865 - val_accuracy: 0.9312 - val_loss: 0.1898
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9806 - loss: 0.0578 - val_accuracy: 0.9316 - val_loss: 0.2060
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9867 - loss: 0.0391 - val_accuracy: 0.9371 - val_loss: 0.2328
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9893 - loss: 0.0315 - val_accuracy: 0.9341 - val_loss: 0.2484
Epoch 8/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9906 - loss: 0.0278 - val_accuracy: 0.

In [23]:
# Simple Feedforward NN with preprocessing setup2
model_sfnn2 = model_sfnn_train(X_train_tfidf3, data_train['label'], X_val_tfidf3, data_val['label'])

Epoch 1/20


d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8629 - loss: 0.3252 - val_accuracy: 0.9294 - val_loss: 0.1859
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9384 - loss: 0.1574 - val_accuracy: 0.9327 - val_loss: 0.1850
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9510 - loss: 0.1235 - val_accuracy: 0.9330 - val_loss: 0.1925
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9666 - loss: 0.0917 - val_accuracy: 0.9345 - val_loss: 0.2125
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9760 - loss: 0.0667 - val_accuracy: 0.9316 - val_loss: 0.2376
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9833 - loss: 0.0503 - val_accuracy: 0.9305 - val_loss: 0.2629
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9875 - loss: 0.0382 - val_accuracy: 0.9338 - val_loss: 0.2815
982/982 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Train Accuracy: 96.18%
Validation A

In [10]:
# Simple Feedforward NN with preprocessing setup3
model_sfnn3 = model_sfnn_train(X_train_emb, data_train['label'], X_val_emb, data_val['label'])

Epoch 1/20


d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9030 - loss: 0.2514 - val_accuracy: 0.9338 - val_loss: 0.1640
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9403 - loss: 0.1519 - val_accuracy: 0.9411 - val_loss: 0.1483
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9535 - loss: 0.1208 - val_accuracy: 0.9458 - val_loss: 0.1370
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9647 - loss: 0.0945 - val_accuracy: 0.9488 - val_loss: 0.1281
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9743 - loss: 0.0728 - val_accuracy: 0.9539 - val_loss: 0.1302
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9796 - loss: 0.0563 - val_accuracy: 0.9521 - val_loss: 0.1382
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9838 - loss: 0.0447 - val_accuracy: 0.9510 - val_loss: 0.1319
Epoch 8/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9875 - loss: 0.0364 - val_accuracy: 0.9528 - val_

In [ ]:
## make predictions on test data
filepath = r".\data\testing_data_lowercase_labels.csv"

#predict_values(model_rf, tfidf_matrix_test, data_test, filepath)
#model_nb2 = model_naives_bayes(X_train_tfidf, data_train['label'], X_val_tfidf, data_val['label'])
predict_values(model_nb2, X_test_tfidf, data_test, filepath)
